In [1]:
# Les visualisations contenues dans ce notebook ont été commentés car elles alourdiraient trop le fichier ce qui rendrait son upload sur le repertoire Github impossible
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import scipy.stats as st
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from datetime import datetime, timedelta

# Connection à la base de données SQLite
conn = sqlite3.connect("olist.db")

# Sélectionne toutes les colonnes de la table "orders"
query = "SELECT * FROM orders;"

# Crée un dataframe avec la table sélectionnée précédemment
df_orders = pd.read_sql_query(query, conn)

query = "SELECT * FROM customers;"
df_customers = pd.read_sql_query(query, conn)

query = "SELECT * FROM geoloc;"
df_geoloc = pd.read_sql_query(query, conn)

query = "SELECT * FROM order_items;"
df_order_items = pd.read_sql_query(query, conn)

query = "SELECT * FROM order_pymts;"
df_order_pymts = pd.read_sql_query(query, conn)

query = "SELECT * FROM order_reviews;"
df_order_reviews = pd.read_sql_query(query, conn)

query = "SELECT * FROM products;"
df_products = pd.read_sql_query(query, conn)

query = "SELECT * FROM sellers;"
df_sellers = pd.read_sql_query(query, conn)

query = "SELECT * FROM translation;"
df_translation = pd.read_sql_query(query, conn)

# Ferme la connection avec la base de données
conn.close()
# Analyse prix des commandes
mean_price_df_order_items = df_order_items["price"].mean()
min_price_df_order_items = df_order_items["price"].min()
max_price_df_order_items = df_order_items["price"].max()

print(f"Mean price of orders: {mean_price_df_order_items}")
print(f"Minimum price of orders: {min_price_df_order_items}")
print(f"Maximum price of orders: {max_price_df_order_items}")
# Analyse type de paiement
payment_type_counts = df_order_pymts['payment_type'].value_counts()

print(payment_type_counts)
# Création du dataframe avec les colonnes utiles
# Fusion de df_orders et df_customers sur la colonne "customer_id"
merged_df = pd.merge(df_orders, df_customers, on='customer_id', how='inner', suffixes=('_orders', '_customers'))

# Fusion du résultat avec df_order_pymts sur la colonne "order_id"
merged_df = pd.merge(merged_df, df_order_pymts, on='order_id', how='inner', suffixes=('_merged', '_order_pymts'))

# Fusion du résultat avec df_order_reviews sur la colonne "order_id"
merged_df = pd.merge(merged_df, df_order_reviews, on='order_id', suffixes=('_merged', '_order_reviews'))

merged_df.head()
# Création d'une nouvelle colonne "order_purchase_timestamp_datetime" de type datetime
merged_df['order_purchase_timestamp_datetime'] = pd.to_datetime(merged_df['order_purchase_timestamp'])
merged_df.head()
# Date la plus ancienne et date la plus récente
earliest_date = merged_df['order_purchase_timestamp_datetime'].min()
latest_date = merged_df['order_purchase_timestamp_datetime'].max()

print(f"Date la plus ancienne : {earliest_date}")
print(f"Date la plus récente : {latest_date}")
missing_values_count = merged_df.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(missing_values_count)
# Suppression des colonnes "review_comment_title" et "review_comment_message"
merged_df = merged_df.drop(['review_comment_title', 'review_comment_message'], axis=1)
# Suppression des lignes avec des valeurs manquantes
merged_df_cleaned = merged_df.dropna()
missing_values_count = merged_df_cleaned.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(missing_values_count)
# Date la plus ancienne et date la plus récente
earliest_date = merged_df_cleaned['order_purchase_timestamp_datetime'].min()
latest_date = merged_df_cleaned['order_purchase_timestamp_datetime'].max()

print(f"Date la plus ancienne : {earliest_date}")
print(f"Date la plus récente : {latest_date}")
# Trier le dataframe en fonction de la colonne 'order_purchase_timestamp_datetime'
merged_df_cleaned.sort_values(by='order_purchase_timestamp_datetime', inplace=True)
merged_df_cleaned.head()


def creation_analyse_rfm(dataframe_periode, end_date):
    """ Fonction permettant la réalisation d'une analyse rfm + average review score
    dataframe_periode : dataframe sur lequel on souhaite réaliser l'analyse rfm
    end_date : date de la fin de la période sur laquelle s'étend le dataframe"""

    # Pour la récence
    # Trouver la date de la commande la plus récente pour chaque customer_unique_id
    latest_order_date_df = dataframe_periode.groupby('customer_unique_id')[
        'order_purchase_timestamp_datetime'].max().reset_index()
    latest_order_date_df.columns = ['customer_unique_id', 'latest_order_date']

    # Calculer la récence en jours jusqu'à la fin de la période spécifiée
    latest_order_date_df['recency'] = (pd.to_datetime(end_date) - latest_order_date_df['latest_order_date']).dt.days

    # Pour la fréquence
    # Calculer la fréquence d'achat par client
    frequency_df = dataframe_periode.groupby('customer_unique_id')['order_id'].nunique().reset_index()
    frequency_df.columns = ['customer_unique_id', 'frequency']

    # Pour le montant
    # Calculer le montant total dépensé par commande pour chaque client_unique_id
    total_amount_spent = dataframe_periode.groupby('customer_unique_id')['payment_value'].sum()

    # Calculer le nombre total de commandes pour chaque client_unique_id
    total_orders = dataframe_periode.groupby('customer_unique_id')['order_id'].nunique()

    # Calculer la moyenne par commande pour chaque client_unique_id
    average_spending_per_order = total_amount_spent / total_orders

    # Créer un DataFrame avec les résultats
    average_spending_df = pd.DataFrame({'customer_unique_id': average_spending_per_order.index,
                                        'average_spending_per_order': average_spending_per_order.values})

    # Pour la moyenne du review_score par client
    average_score_per_customer = merged_df.groupby('customer_unique_id')['review_score'].mean().reset_index()

    # Fusion des DataFrames sur le customer_unique_id
    rfm_df = pd.merge(frequency_df, latest_order_date_df, on='customer_unique_id', how='left')
    rfm_df = pd.merge(rfm_df, average_spending_df, on='customer_unique_id', how='left')
    rfm_df = pd.merge(rfm_df, average_score_per_customer, on='customer_unique_id', how='left')

    # Ajout de la colonne 'order_purchase_timestamp_datetime'
    rfm_df = pd.merge(rfm_df, dataframe_periode[['customer_unique_id', 'order_purchase_timestamp_datetime']],
                      on='customer_unique_id', how='left')

    # Trie du dataframe en fonction de la colonne 'order_purchase_timestamp_datetime'
    rfm_df.sort_values(by='order_purchase_timestamp_datetime', inplace=True)

    return rfm_df


rfm_df_2016_2018 = creation_analyse_rfm(dataframe_periode=merged_df_cleaned, end_date="2018-10-31")
rfm_df_2016_2018.head()
# Supprimer les colonnes non pertinentes
columns_to_exclude = ['customer_unique_id', 'order_purchase_timestamp_datetime', "latest_order_date"]
rfm_df_2016_2018_filtered = rfm_df_2016_2018.drop(columns=columns_to_exclude)

rfm_df_2016_2018_filtered.head()
# Créer un objet StandardScaler
scaler = StandardScaler()

# Standardisation de filtered_combined_df
rfm_df_2016_2018_filtered_standardized = pd.DataFrame(scaler.fit_transform(rfm_df_2016_2018_filtered),
                                                      columns=rfm_df_2016_2018_filtered.columns)

rfm_df_2016_2018_filtered_standardized.head()

Mean price of orders: 120.65373901464716
Minimum price of orders: 0.85
Maximum price of orders: 6735.0
payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64
Date la plus ancienne : 2016-09-04 21:15:19
Date la plus récente : 2018-10-17 17:30:18
Nombre de valeurs manquantes par colonne :
index_orders                             0
order_id                                 0
customer_id                              0
order_status                             0
order_purchase_timestamp                 0
order_approved_at                      171
order_delivered_carrier_date          1861
order_delivered_customer_date         3030
order_estimated_delivery_date            0
index_customers                          0
customer_unique_id                       0
customer_zip_code_prefix                 0
customer_city                            0
customer_state                           0
index_merged         

/var/folders/xt/s64lptzd6js02z9ywj1ydl8m0000gn/T/ipykernel_43366/608484631.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_cleaned.sort_values(by='order_purchase_timestamp_datetime', inplace=True)


,frequency,recency,average_spending_per_order,review_score
0,5.037118,-1.154033,-0.529844,0.468876
1,-0.212799,2.989249,-0.560384,-0.119798
2,-0.212799,2.989249,-0.576413,-2.474491
3,-0.212799,2.989249,-0.492948,-0.904695
4,-0.212799,2.989249,-0.125691,-2.474491


In [2]:
from sklearn.cluster import DBSCAN
from sklearn.model_selection import ParameterGrid

# Grille d'hyperparamètres
param_grid = {
    'eps': [0.1, 0.5, 1.0],
    'min_samples': [5, 10, 20]
}

best_score = -1
best_params = None

# Gridsearch
for params in ParameterGrid(param_grid):
    dbscan = DBSCAN(**params)
    dbscan.fit(rfm_df_2016_2018_filtered_standardized)
    labels = dbscan.labels_
    score = silhouette_score(rfm_df_2016_2018_filtered_standardized, labels)

    if score > best_score:
        best_score = score
        best_params = params

print("Hyperparamètres:", best_params)

Hyperparamètres: {'eps': 1.0, 'min_samples': 10}


In [3]:
# Entraînement d'un modèle DBSCAN

# Créer un objet StandardScaler
scaler_dbscan = StandardScaler()

# Standardisation de filtered_combined_df
rfm_df_2016_2018_filtered_standardized_dbscan = pd.DataFrame(scaler_dbscan.fit_transform(rfm_df_2016_2018_filtered), columns=rfm_df_2016_2018_filtered.columns)

rfm_df_2016_2018_filtered_standardized_dbscan.head()

,frequency,recency,average_spending_per_order,review_score
0,5.037118,-1.154033,-0.529844,0.468876
1,-0.212799,2.989249,-0.560384,-0.119798
2,-0.212799,2.989249,-0.576413,-2.474491
3,-0.212799,2.989249,-0.492948,-0.904695
4,-0.212799,2.989249,-0.125691,-2.474491


In [4]:
# Entraînement
dbscan = DBSCAN(eps=1.0, min_samples=10)

# Clustering
cluster_label_dbscan = dbscan.fit_predict(rfm_df_2016_2018_filtered_standardized_dbscan)

# Etiquettes de cluster
rfm_df_2016_2018_filtered_standardized_dbscan['cluster_label_dbscan'] = cluster_label_dbscan

# Calcul du score de silhouette
silhouette_avg = silhouette_score(rfm_df_2016_2018_filtered_standardized_dbscan, cluster_label_dbscan)
print("Score de silhouette :", silhouette_avg)

Score de silhouette : 0.4351535908387644


In [5]:
# Déstandardisation du dataframe dbscan
rfm_df_2016_2018_filtered_destandardized_dbscan = rfm_df_2016_2018_filtered_standardized_dbscan.copy()

In [6]:
# Suppression de la colonne "cluster_label"
rfm_df_2016_2018_filtered_destandardized_dbscan.drop('cluster_label_dbscan', axis=1, inplace=True)

In [7]:
# Déstandardisation
rfm_df_2016_2018_filtered_destandardized_dbscan = pd.DataFrame(
    scaler_dbscan.inverse_transform(rfm_df_2016_2018_filtered_destandardized_dbscan),
    columns=rfm_df_2016_2018_filtered_destandardized_dbscan.columns)

In [8]:
# Ajout de la colonne 'cluster_label' au dataframe déstandardisé
rfm_df_2016_2018_filtered_destandardized_dbscan['cluster_label_dbscan'] = rfm_df_2016_2018_filtered_standardized_dbscan[
    'cluster_label_dbscan']
rfm_df_2016_2018_filtered_destandardized_dbscan.head()

,frequency,recency,average_spending_per_order,review_score,cluster_label_dbscan
0,3.0,124.0,45.72,4.75,0
1,1.0,757.0,39.09,4.00,1
2,1.0,757.0,35.61,1.00,1
3,1.0,757.0,53.73,3.00,1
4,1.0,757.0,133.46,1.00,1


In [22]:
# Visualisation clusters DBSCAN "recency" par rapport à "average_spending_per_order"
# fig = px.scatter(rfm_df_2016_2018_filtered_destandardized_dbscan,
#                  x='recency',
#                  y='average_spending_per_order',
#                  color=cluster_label_dbscan,
#                  color_continuous_scale='viridis',
#                  opacity=0.5,
#                  labels={'recency': 'Recency', 'average_spending_per_order': 'Average Spending per Order'},
#                  title='DBSCAN Clustering')

# fig.update_layout(
#     xaxis_title='Recency',
#     yaxis_title='Average Spending per Order',
#     title='DBSCAN Clustering'
# )

# fig.show()

In [23]:
# Visualisation clusters DBSCAN "frequency" par rapport à "average_spending_per_order"
# fig = px.scatter(rfm_df_2016_2018_filtered_destandardized_dbscan,
#                  x='frequency',
#                  y='average_spending_per_order',
#                  color=cluster_label_dbscan,
#                  color_continuous_scale='viridis',
#                  opacity=0.5,
#                  labels={'frequency': 'Frequency', 'average_spending_per_order': 'Average Spending per Order'},
#                  title='DBSCAN Clustering')

# fig.update_layout(
#     xaxis_title='Frequency',
#     yaxis_title='Average Spending per Order',
#     title='DBSCAN Clustering'
# )

In [24]:
# Visualisation clusters DBSCAN "average_review_score" par rapport à "average_spending_per_order"
# fig = px.scatter(rfm_df_2016_2018_filtered_destandardized_dbscan,
#                  x='review_score',
#                  y='average_spending_per_order',
#                  color=cluster_label_dbscan,
#                  color_continuous_scale='viridis',
#                  opacity=0.5,
#                  labels={'review_score': 'Average Review Score',
#                          'average_spending_per_order': 'Average Spending per Order'},
#                  title='DBSCAN Clustering')

# fig.update_layout(
#     xaxis_title='Average Review Score',
#     yaxis_title='Average Spending per Order',
#     title='DBSCAN Clustering'
# )

# fig.show()

In [25]:
# Palette de couleurs personnalisée
# custom_palette = sns.color_palette("husl", n_colors=len(
#     rfm_df_2016_2018_filtered_destandardized_dbscan['cluster_label_dbscan'].unique()))

# Pairplot des caractéristiques
# sns.set(style="ticks")
# sns.pairplot(rfm_df_2016_2018_filtered_destandardized_dbscan, hue='cluster_label_dbscan', diag_kind='kde', palette=custom_palette)
# plt.show()

In [26]:
# Boxplot pour la variable "recency" du modèle DBSCAN
# fig = px.box(rfm_df_2016_2018_filtered_destandardized_dbscan, x='cluster_label_dbscan', y='recency',
#              color='cluster_label_dbscan', points='all', title='Boxplot de Recency par Cluster Modèle DBSCAN')
# fig.show()

In [27]:
# Boxplot pour la variable "frequency" du modèle DBSCAN
# fig = px.box(rfm_df_2016_2018_filtered_destandardized_dbscan, x='cluster_label_dbscan', y='frequency',
#              color='cluster_label_dbscan', points='all', title='Boxplot de Frequency par Cluster Modèle DBSCAN')
# fig.show()

In [28]:
# Boxplot pour la variable "average_spending_per_order" du modèle DBSCAN
# fig = px.box(rfm_df_2016_2018_filtered_destandardized_dbscan, x='cluster_label_dbscan', y='average_spending_per_order',
#              color='cluster_label_dbscan', points='all',
#              title='Boxplot de average_spending_per_order par Cluster Modèle DBSCAN')
# fig.show()

In [29]:
# Boxplot pour la variable average review score du modèle DBSCAN
# fig = px.box(rfm_df_2016_2018_filtered_destandardized_dbscan, x='cluster_label_dbscan', y='review_score',
#              color='cluster_label_dbscan', points='all',
#              title='Boxplot de average review score par Cluster Modèle DBSCAN')
# fig.show()

In [17]:
# Partie maintenance du modèle
def creation_data_frame_par_periode(start_date, end_date, merged_df_cleaned):
    """Crée un dataframe pour la période spécifiée :
    start_date : début de la période souhaitée
    end_date : fin de la période souhaitée
     merge_df_cleaned = dataframe contenant les données sur lesquels on souhaite travailler"""

    # Crée un dataframe pour la période spécifiée
    filtered_df = merged_df_cleaned[(merged_df_cleaned['order_purchase_timestamp_datetime'] >= start_date) & (
            merged_df_cleaned['order_purchase_timestamp_datetime'] <= end_date)].copy()

    return filtered_df

In [18]:
def creation_analyse_rfm(dataframe_periode, end_date):
    """Fonction réalisant une analyse rfm + average review score sur le dataframe spécifié
    dataframe_periode : dataframe de la période souhaitée sur lequel on souhaite réaliser l'analyse
    end_date : dernier jour de la période souhaitée
     """
    # Pour la récence
    # Trouve la date de la commande la plus récente pour chaque customer_unique_id
    latest_order_date_df = dataframe_periode.groupby('customer_unique_id')[
        'order_purchase_timestamp_datetime'].max().reset_index()
    latest_order_date_df.columns = ['customer_unique_id', 'latest_order_date']

    # Calcul la récence en jours jusqu'à la fin de la période spécifiée
    latest_order_date_df['recency'] = (pd.to_datetime(end_date) - latest_order_date_df['latest_order_date']).dt.days

    # Pour la fréquence
    # Calcul la fréquence d'achat par client
    frequency_df = dataframe_periode.groupby('customer_unique_id')['order_id'].nunique().reset_index()
    frequency_df.columns = ['customer_unique_id', 'frequency']

    # Pour le montant
    # Calcul le montant total dépensé par commande pour chaque client_unique_id
    total_amount_spent = dataframe_periode.groupby('customer_unique_id')['payment_value'].sum()

    # Calcul le nombre total de commandes pour chaque client_unique_id
    total_orders = dataframe_periode.groupby('customer_unique_id')['order_id'].nunique()

    # Calcul la moyenne par commande pour chaque client_unique_id
    average_spending_per_order = total_amount_spent / total_orders

    # Crée un DataFrame avec les résultats
    average_spending_df = pd.DataFrame({'customer_unique_id': average_spending_per_order.index,
                                        'average_spending_per_order': average_spending_per_order.values})

    # Pour la moyenne du review_score par client
    average_score_per_customer = merged_df.groupby('customer_unique_id')['review_score'].mean().reset_index()

    # Fusion des DataFrames sur 'customer_unique_id'
    rfm_df = pd.merge(frequency_df, latest_order_date_df, on='customer_unique_id', how='left')
    rfm_df = pd.merge(rfm_df, average_spending_df, on='customer_unique_id', how='left')
    rfm_df = pd.merge(rfm_df, average_score_per_customer, on='customer_unique_id', how='left')

    # Ajoute la colonne 'order_purchase_timestamp_datetime'
    rfm_df = pd.merge(rfm_df, dataframe_periode[['customer_unique_id', 'order_purchase_timestamp_datetime']],
                      on='customer_unique_id', how='left')

    # Trie le dataframe en fonction de la colonne 'order_purchase_timestamp_datetime'
    rfm_df.sort_values(by='order_purchase_timestamp_datetime', inplace=True)

    return rfm_df

In [19]:
# Maintenance
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# Date d'initialisation
init_date = datetime(2018, 1, 1)

# Création du dataframe pour les données de 2016 à 2017
df_1 = creation_data_frame_par_periode(start_date="2015-01-01", end_date=init_date, merged_df_cleaned=merged_df_cleaned)

# Création des analyses rfm pour le dataframe de référence
rfm_1 = creation_analyse_rfm(dataframe_periode=df_1, end_date=init_date)

# Suppression des colonnes non pertinentes
columns_to_exclude = ['customer_unique_id', "latest_order_date", "order_purchase_timestamp_datetime"]

# Entraînement du modèle Kmeans pour le dataframe de référence
kmeans_1 = KMeans(n_clusters=5, random_state=42).fit(rfm_1.drop(columns=columns_to_exclude))

# Initialisation de la liste pour stocker les scores ARI
ari_scores = []

# Boucle créant un dataframe par mois jusqu'au mois d'août 2018 (date de la dernière commande), puis création des analyses rfm, entraînement des modèles et comparaison de la similarité des modèles de 2016-2017
for i in range(8):
    end_date = init_date + timedelta(days=i * 30)
    df_2 = creation_data_frame_par_periode(start_date="2015-01-01", end_date=end_date,
                                           merged_df_cleaned=merged_df_cleaned)

    # Création des analyses rfm pour le dataframe actuel
    rfm_2 = creation_analyse_rfm(dataframe_periode=df_2, end_date=end_date)

    # Entraînement du modèle Kmeans pour le dataframe actuel
    kmeans_2 = KMeans(n_clusters=5, random_state=42).fit(rfm_2.drop(columns=columns_to_exclude))

    # Prédictions du modèle sur les nouvelles données
    predict_reference = kmeans_1.predict(rfm_2.drop(columns=columns_to_exclude))
    predict_2 = kmeans_2.predict(rfm_2.drop(columns=columns_to_exclude))

    # Comparaison des modèles grâce à l'Adjusted Rand Score et stockage du score dans la liste
    ari = adjusted_rand_score(predict_reference, predict_2)
    ari_scores.append(ari)

# Affichage des scores ARI pour chaque itération
for i, ari in enumerate(ari_scores):
    print(f"Iteration {i + 1} - Adjusted Rand Score entre le modèle de référence et le modèle {i + 1}: {ari}")

/Users/tom/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/tom/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/tom/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/tom/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/tom/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: F

Iteration 1 - Adjusted Rand Score entre le modèle de référence et le modèle 1: 1.0
Iteration 2 - Adjusted Rand Score entre le modèle de référence et le modèle 2: 0.9140863725383442
Iteration 3 - Adjusted Rand Score entre le modèle de référence et le modèle 3: 0.7282948230659595
Iteration 4 - Adjusted Rand Score entre le modèle de référence et le modèle 4: 0.7328617472435557
Iteration 5 - Adjusted Rand Score entre le modèle de référence et le modèle 5: 0.5665507808208012
Iteration 6 - Adjusted Rand Score entre le modèle de référence et le modèle 6: 0.47906798686757285
Iteration 7 - Adjusted Rand Score entre le modèle de référence et le modèle 7: 0.40247458415812654
Iteration 8 - Adjusted Rand Score entre le modèle de référence et le modèle 8: 0.3334101815987905
